# All Call Graph

In [ ]:
import networkx as nx

from qualtran_dev_tools.bloq_finder import get_bloq_examples, get_bloq_classes
from qualtran_dev_tools.all_call_graph import get_all_call_graph, call_graph_to_class_graph, ClassGraphGraphviz

from qualtran.drawing import show_call_graph
from IPython.display import SVG, display, HTML

In [ ]:
bes = get_bloq_examples()

In [ ]:
g = get_all_call_graph(bes)

from qualtran.bloqs.basic_gates import TGate, Toffoli

g.remove_nodes_from([TGate(), Toffoli()])

In [ ]:
leafs = []
heads = []
for bloq in nx.topological_sort(g):
    if not g.succ[bloq]:
        leafs.append(bloq)
        continue
    if not g.pred[bloq]:
        heads.append(bloq)
    subg = nx.subgraph(g, {bloq} | set(g.successors(bloq)))
    show_call_graph(subg)
    display(HTML('<hr/>'))

In [ ]:
for leaf in leafs:
    print(leaf.pretty_name())

In [ ]:
for head in heads:
    subg = nx.subgraph(g, {head} | set(nx.descendants(g, head)))
    show_call_graph(subg)
    display(HTML('<hr/>'))
    

In [ ]:
import base64
_html_template='<img width="{}" src="data:image/svg+xml;base64,{}" >'

def svg_to_fixed_width_html_image(svg, width="100%"):
    text = _html_template.format(width, base64.b64encode(svg).decode())
    return HTML(text)

In [ ]:
from qualtran.drawing import GraphvizCounts
svg_to_fixed_width_html_image(GraphvizCounts(g).get_svg_bytes(), width="200%")

In [ ]:
for x in nx.simple_cycles(g):
    print(x)

In [ ]:
# with open('all-call.svg', 'wb') as f:
#     f.write(GraphvizCounts(g).get_svg_bytes())

In [ ]:
# import pydot
# def get_graph(self):
#     """Get the pydot graph."""
#     graph = pydot.Dot('classes', graph_type='digraph', rankdir='TB', ranksep=0.8)
#     self.add_nodes(graph)
#     self.add_edges(graph)
#     return graph
# ClassGraphGraphviz.get_graph = get_graph

In [ ]:
g2 = call_graph_to_class_graph(g)
for x in nx.simple_cycles(g2):
    print(x)
svg_to_fixed_width_html_image(ClassGraphGraphviz(g2).get_svg_bytes(), width="200%")

In [ ]:
# with open('class-graph.svg', 'wb') as f:
#     f.write(ClassGraphGraphviz(g2).get_svg_bytes())

In [ ]:
g3 = nx.subgraph(g2, max(nx.weakly_connected_components(g2), key=len))
ClassGraphGraphviz(g3).get_svg()

In [ ]:
bcls = get_bloq_classes()

In [ ]:
import pandas as pd

records = []
for bcl in bcls:
    records += [{
        'bloq_cls': bcl.__name__,
        'exists': bcl in g2.nodes,
        'connected': bcl in g3.nodes,
    }]

df = pd.DataFrame(records)

def colors(x):
    if x:
        return 'background-color: lightgreen'
    return 'background-color:pink'

df.style.applymap(colors, ['exists', 'connected'])